# Discovering Sentinel-2 products

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import visualize_S2_products 

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

import geopandas as gpd
import pandas as pd
pd.set_option('display.max_colwidth', None)
import folium
import folium.plugins
import logging
import numpy as np

import shapely
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
# file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])
shp = gpd.read_file(file_aoi).geometry[0]

# open credentials
with open("/data/creds.json") as f:
    creds = json.load(f)

# search for products intersecting geometry
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    platformname="Sentinel-2",
    date=("20230901", "20230920"),
    producttype="S2MSI1C",
    #  orbitdirection='ASCENDING',
    limit=50,
)

# make a geodataframe for visualization
gdf = api.to_geodataframe(products).reset_index(names='id')
dates = np.unique(gdf.beginposition.astype(str)).tolist()
print(f"Unique dates:\n{"\n".join(dates)}")

In [ ]:
visualize_S2_products(gdf, dates[7], gj_aoi)

## Visualize by groups

In [ ]:
visualize_S2_products(gdf, gj_aoi)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [1, 5]
ids = gdf['id'][indices_to_download]
# dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')
dl = api.download_all(ids, directory_path='/data/S1/')